In [1]:
# Cell 1: Setup and Configuration
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any

FHIR_BASE = "http://localhost:32783/csp/healthshare/demo/fhir/r4"
FHIR_AUTH = ("_SYSTEM", "ISCDEMO")
FHIR_HEADERS = {
    "Content-Type": "application/fhir+json; charset=utf-8",
    "Accept": "application/fhir+json"
}
SYSTEM_BASE = "https://hackbrno.vercel.app"

In [2]:
# Cell 2: Core FHIR Helper Functions

def send_to_fhir(resource: Dict[str, Any]) -> Dict[str, Any]:
    """Send a single resource or bundle to FHIR server."""
    if resource.get("resourceType") == "Bundle":
        url = FHIR_BASE
    else:
        resource_type = resource["resourceType"]
        resource_id = resource.get("id", "")
        url = f"{FHIR_BASE}/{resource_type}/{resource_id}" if resource_id else f"{FHIR_BASE}/{resource_type}"
    
    method = "PUT" if resource.get("id") else "POST"
    response = requests.request(method, url, json=resource, headers=FHIR_HEADERS, auth=FHIR_AUTH)
    
    return {"status": response.status_code, "response": response.json() if response.text else {}}


def create_bundle(entries: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Wrap multiple resources in a transaction bundle."""
    return {
        "resourceType": "Bundle",
        "type": "transaction",
        "entry": entries
    }


def bundle_entry(resource: Dict[str, Any], method: str = "POST") -> Dict[str, Any]:
    """Create a bundle entry for a resource."""
    entry = {"resource": resource, "request": {"method": method, "url": resource["resourceType"]}}
    if resource.get("id"):
        entry["request"]["url"] = f"{resource['resourceType']}/{resource['id']}"
        entry["request"]["method"] = "PUT"
    return entry

In [3]:
# Cell 3: Patient Resource

def create_patient(patient_id: str) -> Dict[str, Any]:
    """Create a Patient resource."""
    return {
        "resourceType": "Patient",
        "id": patient_id,
        "identifier": [{
            "system": f"{SYSTEM_BASE}/patient-id",
            "value": patient_id
        }]
    }

In [4]:
# Cell 4: Diagnosis with Date (Condition)

def create_diagnosis(
    patient_id: str,
    diagnosis_date: str,  # Format: YYYY-MM-DD
    diagnosis_text: str,
    icd10_code: Optional[str] = None,
    condition_id: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create a Condition resource for diagnosis.
    datum stanovení diagnózy = onsetDateTime
    """
    condition = {
        "resourceType": "Condition",
        "clinicalStatus": {
            "coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-clinical", "code": "active"}]
        },
        "verificationStatus": {
            "coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-ver-status", "code": "confirmed"}]
        },
        "code": {"text": diagnosis_text},
        "subject": {"reference": f"Patient/{patient_id}"},
        "onsetDateTime": diagnosis_date
    }
    
    if icd10_code:
        condition["code"]["coding"] = [{"system": "http://hl7.org/fhir/sid/icd-10", "code": icd10_code}]
    
    if condition_id:
        condition["id"] = condition_id
    
    return condition

In [6]:
# Cell 5: TNM Classification (Observation)

def create_tnm(
    patient_id: str,
    tnm_value: str,  # e.g., "cT2N0M0" or "pT2N0M0"
    date: str,
    is_pathological: bool = False,
    condition_ref: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create TNM staging Observation.
    klinická TNM: is_pathological=False
    patologická TNM: is_pathological=True
    """
    tnm_type = "pathological" if is_pathological else "clinical"
    loinc_code = "21902-2" if is_pathological else "21908-9"
    
    obs = {
        "resourceType": "Observation",
        "status": "final",
        "category": [{"coding": [{"system": "http://terminology.hl7.org/CodeSystem/observation-category", "code": "survey"}]}],
        "code": {
            "coding": [{"system": "http://loinc.org", "code": loinc_code, "display": f"Stage group.{tnm_type} Cancer"}],
            "text": f"TNM klasifikace ({tnm_type})"
        },
        "subject": {"reference": f"Patient/{patient_id}"},
        "effectiveDateTime": date,
        "valueString": tnm_value
    }
    
    if condition_ref:
        obs["focus"] = [{"reference": condition_ref}]
    
    return obs

In [7]:
# Cell 6: Biomarkers - ER, PR, HER2 (Observation)

def create_biomarker(
    patient_id: str,
    marker: str,  # "ER", "PR", "HER2", "Ki67"
    value: Any,   # percentage (float) for ER/PR/Ki67, or "positive"/"negative" for HER2
    date: str,
    note: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create biomarker Observation.
    vyšetření hormonálních receptorů (ER, PR) a HER2
    """
    loinc_codes = {
        "ER": ("16112-5", "Estrogen receptor"),
        "PR": ("16113-3", "Progesterone receptor"),
        "HER2": ("48676-1", "HER2"),
        "Ki67": ("29593-1", "Ki-67")
    }
    
    code, display = loinc_codes.get(marker, (None, marker))
    
    obs = {
        "resourceType": "Observation",
        "status": "final",
        "code": {
            "coding": [{"system": "http://loinc.org", "code": code, "display": display}] if code else [],
            "text": f"{marker} status"
        },
        "subject": {"reference": f"Patient/{patient_id}"},
        "effectiveDateTime": date
    }
    
    # Value handling
    if isinstance(value, (int, float)):
        obs["valueQuantity"] = {"value": value, "unit": "%", "system": "http://unitsofmeasure.org", "code": "%"}
    elif value in ("positive", "negative"):
        obs["valueCodeableConcept"] = {"coding": [{"display": value.capitalize()}], "text": value}
    else:
        obs["valueString"] = str(value)
    
    if note:
        obs["note"] = [{"text": note}]
    
    return obs

In [8]:
# Cell 7: External Treatment (Procedure)

def create_external_treatment(
    patient_id: str,
    treatment_description: str,
    date: str,
    external_institution: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create Procedure for treatment outside MOÚ.
    léčba mimo MOÚ
    """
    return {
        "resourceType": "Procedure",
        "status": "completed",
        "code": {"text": treatment_description},
        "subject": {"reference": f"Patient/{patient_id}"},
        "performedDateTime": date,
        "performer": [{
            "actor": {"display": external_institution or "Externí pracoviště"},
            "onBehalfOf": {"display": "Mimo MOÚ"}
        }],
        "note": [{"text": f"Léčba provedena mimo MOÚ: {treatment_description}"}]
    }

In [9]:
# Cell 8: Progression (ClinicalImpression)

def create_progression(
    patient_id: str,
    date: str,
    description: str,
    condition_ref: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create ClinicalImpression for disease progression.
    progrese
    """
    impression = {
        "resourceType": "ClinicalImpression",
        "status": "completed",
        "subject": {"reference": f"Patient/{patient_id}"},
        "effectiveDateTime": date,
        "finding": [{
            "itemCodeableConcept": {
                "coding": [{"system": "http://snomed.info/sct", "code": "246450006", "display": "Disease progression"}],
                "text": "Progrese onemocnění"
            },
            "basis": description
        }],
        "summary": description
    }
    
    if condition_ref:
        impression["problem"] = [{"reference": condition_ref}]
    
    return impression

In [10]:
# Cell 9: Recurrence (Condition)

def create_recurrence(
    patient_id: str,
    date: str,
    description: str,
    original_condition_ref: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create Condition for recurrence.
    recidiva
    """
    condition = {
        "resourceType": "Condition",
        "clinicalStatus": {
            "coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-clinical", "code": "recurrence"}]
        },
        "code": {
            "coding": [{"system": "http://snomed.info/sct", "code": "277022003", "display": "Recurrent malignant neoplasm"}],
            "text": description
        },
        "subject": {"reference": f"Patient/{patient_id}"},
        "onsetDateTime": date
    }
    
    if original_condition_ref:
        condition["extension"] = [{
            "url": "http://hl7.org/fhir/StructureDefinition/condition-related",
            "valueReference": {"reference": original_condition_ref}
        }]
    
    return condition

In [11]:
# Cell 10: Metastasis (Condition)

def create_metastasis(
    patient_id: str,
    site: str,  # "liver", "bone", "lung", "brain", or custom
    date: str,
    primary_condition_ref: Optional[str] = None,
    description: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create Condition for distant metastasis.
    výskyt vzdálených metastáz
    """
    site_mapping = {
        "liver": ("C78.7", "10200004", "Játra"),
        "bone": ("C79.5", "421060004", "Kost"),
        "lung": ("C78.0", "39607008", "Plíce"),
        "brain": ("C79.3", "12738006", "Mozek")
    }
    
    icd_code, snomed_code, display = site_mapping.get(site.lower(), ("C79.9", None, site))
    
    condition = {
        "resourceType": "Condition",
        "clinicalStatus": {
            "coding": [{"system": "http://terminology.hl7.org/CodeSystem/condition-clinical", "code": "active"}]
        },
        "category": [{
            "coding": [{"system": "http://snomed.info/sct", "code": "94222008", "display": "Secondary malignant neoplasm"}]
        }],
        "code": {
            "coding": [{"system": "http://hl7.org/fhir/sid/icd-10", "code": icd_code}],
            "text": description or f"Metastáza - {display}"
        },
        "subject": {"reference": f"Patient/{patient_id}"},
        "onsetDateTime": date
    }
    
    if snomed_code:
        condition["bodySite"] = [{"coding": [{"system": "http://snomed.info/sct", "code": snomed_code}], "text": display}]
    
    if primary_condition_ref:
        condition["extension"] = [{
            "url": "http://hl7.org/fhir/StructureDefinition/condition-dueTo",
            "valueReference": {"reference": primary_condition_ref}
        }]
    
    return condition

In [12]:
# Cell 11: Complete Patient Ingestion Example

def ingest_patient_data(
    patient_id: str,
    diagnosis_date: str,
    diagnosis_text: str,
    clinical_tnm: Optional[str] = None,
    pathological_tnm: Optional[str] = None,
    tnm_date: Optional[str] = None,
    er_value: Optional[float] = None,
    pr_value: Optional[float] = None,
    her2_status: Optional[str] = None,
    biomarker_date: Optional[str] = None,
    external_treatments: Optional[List[Dict]] = None,  # [{"description": ..., "date": ..., "institution": ...}]
    progressions: Optional[List[Dict]] = None,  # [{"date": ..., "description": ...}]
    recurrence: Optional[Dict] = None,  # {"date": ..., "description": ...}
    metastases: Optional[List[Dict]] = None,  # [{"site": ..., "date": ...}]
    icd10_code: Optional[str] = None
) -> Dict[str, Any]:
    """
    Ingest all data for a patient in logical order.
    Returns the server response.
    """
    entries = []
    condition_id = f"condition-{patient_id}-primary"
    condition_ref = f"Condition/{condition_id}"
    
    # 1. Patient (first)
    entries.append(bundle_entry(create_patient(patient_id)))
    
    # 2. Primary Diagnosis
    entries.append(bundle_entry(create_diagnosis(
        patient_id, diagnosis_date, diagnosis_text, icd10_code, condition_id
    )))
    
    # 3. TNM Classification
    if clinical_tnm:
        entries.append(bundle_entry(create_tnm(
            patient_id, clinical_tnm, tnm_date or diagnosis_date, is_pathological=False, condition_ref=condition_ref
        )))
    if pathological_tnm:
        entries.append(bundle_entry(create_tnm(
            patient_id, pathological_tnm, tnm_date or diagnosis_date, is_pathological=True, condition_ref=condition_ref
        )))
    
    # 4. Biomarkers (ER, PR, HER2)
    marker_date = biomarker_date or diagnosis_date
    if er_value is not None:
        entries.append(bundle_entry(create_biomarker(patient_id, "ER", er_value, marker_date)))
    if pr_value is not None:
        entries.append(bundle_entry(create_biomarker(patient_id, "PR", pr_value, marker_date)))
    if her2_status:
        entries.append(bundle_entry(create_biomarker(patient_id, "HER2", her2_status, marker_date)))
    
    # 5. External Treatments
    for tx in (external_treatments or []):
        entries.append(bundle_entry(create_external_treatment(
            patient_id, tx["description"], tx["date"], tx.get("institution")
        )))
    
    # 6. Progressions
    for prog in (progressions or []):
        entries.append(bundle_entry(create_progression(
            patient_id, prog["date"], prog["description"], condition_ref
        )))
    
    # 7. Recurrence
    if recurrence:
        entries.append(bundle_entry(create_recurrence(
            patient_id, recurrence["date"], recurrence["description"], condition_ref
        )))
    
    # 8. Metastases
    for met in (metastases or []):
        entries.append(bundle_entry(create_metastasis(
            patient_id, met["site"], met["date"], condition_ref, met.get("description")
        )))
    
    # Send bundle
    bundle = create_bundle(entries)
    return send_to_fhir(bundle)

In [13]:
# Cell 12: Test with HACK01 Patient Data

result = ingest_patient_data(
    patient_id="HACK01",
    diagnosis_date="2022-10-05",
    diagnosis_text="Invazivní mucinózní karcinom pravého prsu",
    icd10_code="C50.4",
    clinical_tnm="cT2N0M0",
    pathological_tnm="pT2 (23 mm) pN0(i-)(sn) M0, L0, V0, R0",
    tnm_date="2022-12-02",
    er_value=100.0,
    pr_value=10.0,
    her2_status="negative",
    biomarker_date="2022-10-05"
)

print(f"Status: {result['status']}")
print(f"Response: {json.dumps(result['response'], indent=2)[:500]}...")

Status: 200
Response: {
  "resourceType": "Bundle",
  "id": "2ae6680e-cd50-11f0-8e7b-0242ac120002",
  "type": "transaction-response",
  "timestamp": "2025-11-29T18:21:04Z",
  "entry": [
    {
      "response": {
        "status": "201",
        "location": "http://localhost:32783/csp/healthshare/demo/fhir/r4/Patient/HACK01",
        "etag": "W/\"1\"",
        "lastModified": "2025-11-29T18:21:04Z"
      }
    },
    {
      "response": {
        "status": "201",
        "location": "http://localhost:32783/csp/healths...


In [14]:
# Cell 13: Test with HACK03 Patient (with metastases)

result = ingest_patient_data(
    patient_id="HACK03",
    diagnosis_date="2022-10-04",
    diagnosis_text="HER2 pozitivní karcinom levého prsu",
    icd10_code="C50.4",
    her2_status="positive",
    biomarker_date="2022-10-04",
    metastases=[
        {"site": "liver", "date": "2022-10-04"},
        {"site": "bone", "date": "2022-10-04", "description": "Mnohočetné kostní metastázy - páteř, pánev"}
    ],
    progressions=[
        {"date": "2023-05-24", "description": "Diseminace s postižením mozkového parenchymu"}
    ]
)

print(f"Status: {result['status']}")

Status: 200


In [18]:
# Cell 14: Verify Data in Database

def get_patient_data(patient_id: str) -> Dict[str, Any]:
    """Retrieve all data for a patient."""
    response = requests.get(
        f"{FHIR_BASE}/Patient/{patient_id}/$everything",
        headers=FHIR_HEADERS,
        auth=FHIR_AUTH
    )
    return response.json()

# Check HACK01
data = get_patient_data("HACK01")
print(f"Total resources for HACK03: {data.get('total', len(data.get('entry', [])))}")

Total resources for HACK03: 7


In [20]:
# Cell 15: Get Patient Summary - All Attributes and Values

def get_patient_summary(patient_id: str) -> Dict[str, Any]:
    """
    Retrieve and format all data for a patient.
    Returns structured summary of all attributes.
    """
    # Get all resources for patient
    response = requests.get(
        f"{FHIR_BASE}/Patient/{patient_id}/$everything",
        headers=FHIR_HEADERS,
        auth=FHIR_AUTH
    )
    
    if response.status_code != 200:
        return {"error": f"Failed to fetch patient: {response.status_code}"}
    
    bundle = response.json()
    entries = bundle.get("entry", [])
    
    # Initialize summary structure
    summary = {
        "patient_id": patient_id,
        "diagnosis": None,
        "diagnosis_date": None,
        "clinical_tnm": None,
        "pathological_tnm": None,
        "biomarkers": {},
        "external_treatments": [],
        "progressions": [],
        "recurrence": None,
        "metastases": [],
        "raw_resources": {"Patient": 0, "Condition": 0, "Observation": 0, "Procedure": 0, "ClinicalImpression": 0}
    }
    
    for entry in entries:
        resource = entry.get("resource", {})
        resource_type = resource.get("resourceType")
        
        # Count resources
        if resource_type in summary["raw_resources"]:
            summary["raw_resources"][resource_type] += 1
        
        # Parse Patient
        if resource_type == "Patient":
            summary["patient_id"] = resource.get("id")
        
        # Parse Conditions (diagnosis, recurrence, metastases)
        elif resource_type == "Condition":
            code_text = resource.get("code", {}).get("text", "")
            clinical_status = resource.get("clinicalStatus", {}).get("coding", [{}])[0].get("code", "")
            onset = resource.get("onsetDateTime")
            
            # Check if it's a metastasis
            category = resource.get("category", [{}])[0].get("coding", [{}])[0].get("code", "")
            if category == "94222008" or "Metastáza" in code_text:
                body_site = resource.get("bodySite", [{}])[0].get("text", "Unknown")
                summary["metastases"].append({
                    "site": body_site,
                    "date": onset,
                    "description": code_text
                })
            # Check if it's a recurrence
            elif clinical_status == "recurrence":
                summary["recurrence"] = {"date": onset, "description": code_text}
            # Primary diagnosis
            else:
                summary["diagnosis"] = code_text
                summary["diagnosis_date"] = onset
                icd_code = resource.get("code", {}).get("coding", [{}])[0].get("code")
                if icd_code:
                    summary["icd10_code"] = icd_code
        
        # Parse Observations (TNM, biomarkers)
        elif resource_type == "Observation":
            code = resource.get("code", {})
            code_text = code.get("text", "")
            loinc = code.get("coding", [{}])[0].get("code", "")
            
            # Get value
            value = None
            if "valueString" in resource:
                value = resource["valueString"]
            elif "valueQuantity" in resource:
                value = resource["valueQuantity"].get("value")
            elif "valueCodeableConcept" in resource:
                value = resource["valueCodeableConcept"].get("text")
            
            date = resource.get("effectiveDateTime")
            
            # TNM staging
            if loinc == "21908-9":  # Clinical TNM
                summary["clinical_tnm"] = {"value": value, "date": date}
            elif loinc == "21902-2":  # Pathological TNM
                summary["pathological_tnm"] = {"value": value, "date": date}
            # Biomarkers
            elif loinc == "16112-5":  # ER
                summary["biomarkers"]["ER"] = {"value": value, "date": date}
            elif loinc == "16113-3":  # PR
                summary["biomarkers"]["PR"] = {"value": value, "date": date}
            elif loinc == "48676-1":  # HER2
                summary["biomarkers"]["HER2"] = {"value": value, "date": date}
            elif loinc == "29593-1":  # Ki67
                summary["biomarkers"]["Ki67"] = {"value": value, "date": date}
        
        # Parse Procedures (external treatments)
        elif resource_type == "Procedure":
            performer = resource.get("performer", [{}])[0].get("actor", {}).get("display", "Unknown")
            summary["external_treatments"].append({
                "description": resource.get("code", {}).get("text", ""),
                "date": resource.get("performedDateTime"),
                "institution": performer
            })
        
        # Parse ClinicalImpressions (progressions)
        elif resource_type == "ClinicalImpression":
            summary["progressions"].append({
                "date": resource.get("effectiveDateTime"),
                "description": resource.get("summary", "")
            })
    
    return summary


def print_patient_summary(patient_id: str):
    """Print formatted patient summary."""
    s = get_patient_summary(patient_id)
    
    if "error" in s:
        print(s["error"])
        return
    
    print(f"{'='*60}")
    print(f"PATIENT: {s['patient_id']}")
    print(f"{'='*60}")
    
    print(f"\n📋 DIAGNOSIS")
    print(f"   Text: {s['diagnosis']}")
    print(f"   Date: {s['diagnosis_date']}")
    if s.get('icd10_code'):
        print(f"   ICD-10: {s['icd10_code']}")
    
    print(f"\n📊 TNM CLASSIFICATION")
    if s['clinical_tnm']:
        print(f"   Clinical:     {s['clinical_tnm']['value']} ({s['clinical_tnm']['date']})")
    if s['pathological_tnm']:
        print(f"   Pathological: {s['pathological_tnm']['value']} ({s['pathological_tnm']['date']})")
    
    print(f"\n🧬 BIOMARKERS")
    for marker, data in s['biomarkers'].items():
        unit = "%" if isinstance(data['value'], (int, float)) else ""
        print(f"   {marker}: {data['value']}{unit} ({data['date']})")
    
    if s['external_treatments']:
        print(f"\n🏥 EXTERNAL TREATMENTS (mimo MOÚ)")
        for tx in s['external_treatments']:
            print(f"   - {tx['description']} @ {tx['institution']} ({tx['date']})")
    
    if s['progressions']:
        print(f"\n📈 PROGRESSIONS")
        for prog in s['progressions']:
            print(f"   - {prog['date']}: {prog['description']}")
    
    if s['recurrence']:
        print(f"\n🔄 RECURRENCE")
        print(f"   Date: {s['recurrence']['date']}")
        print(f"   Description: {s['recurrence']['description']}")
    
    if s['metastases']:
        print(f"\n🎯 METASTASES")
        for met in s['metastases']:
            print(f"   - {met['site']}: {met['description']} ({met['date']})")
    
    print(f"\n📦 RESOURCE COUNTS")
    for rtype, count in s['raw_resources'].items():
        if count > 0:
            print(f"   {rtype}: {count}")

In [21]:
print_patient_summary("HACK01")

PATIENT: HACK01

📋 DIAGNOSIS
   Text: Invazivní mucinózní karcinom pravého prsu
   Date: 2022-10-05
   ICD-10: C50.4

📊 TNM CLASSIFICATION
   Clinical:     cT2N0M0 (2022-12-02)
   Pathological: pT2 (23 mm) pN0(i-)(sn) M0, L0, V0, R0 (2022-12-02)

🧬 BIOMARKERS
   ER: 100.0% (2022-10-05)
   PR: 10.0% (2022-10-05)
   HER2: negative (2022-10-05)

📦 RESOURCE COUNTS
   Patient: 1
   Condition: 1
   Observation: 5
